Projet

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json

In [2]:
headers= {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
    
}

links = []

for  x in range(1,4):
    page = requests.get(f'https://www.marmiton.org/recettes?type=platprincipal&page={x}')
    soup = bs(page.text, "lxml")
    
    homePages = soup.find_all("div",{"class":"recipe-card"})
    for homePage in homePages:
        href_tags = homePage.find("a").get('href')
    
        links.append(href_tags)
    
#print(len(links))
    

# Extract recipe informations

In [3]:
recipes = []
for link in links:
#url = 'https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'
    res = requests.get(link, headers=headers)
    soup = bs(res.content, 'lxml')

    title = soup.find('h1', class_="main-title show-more").text.strip()


    items = soup.find_all('span', class_="ingredient-name show-icon")
    ingredient = []
    for i in items:
        ingredient.append(i.getText().strip())

    #steps = []
    
    #Cooking_steps = soup.find_all('div', class_="recipe-step-list__container")
    #for i in Cooking_steps:
        #steps.append(i.getText().strip())
        
    recipe ={
        'title' : title,
        'ingredient': ingredient 
    }
    
    recipes.append(recipe)
  
    #for i in steps:
        #i.replace('\n','')
        


In [4]:
a = json.dumps(recipes, indent = 4)
l = json.loads(a)

#print(l)

In [5]:
from pymongo import MongoClient

myclient = MongoClient('mongod',27017)
db = myclient["projet"]
collection=db["Nosql"]
collection.insert_many(l)

In [6]:
cur = collection.find({})

for res in cur:
    print(res)


{'_id': ObjectId('60888cb290722cacd3a9f4b8'), 'title': 'Quiche lorraine', 'ingredient': ['poivre', 'sel', 'muscade', 'lait', 'lardons', 'pâte brisée', 'beurre', 'oeuf', 'crème fraîche']}
{'_id': ObjectId('60888cb290722cacd3a9f4b9'), 'title': 'Tomates farcies au Thermomix', 'ingredient': ['sel', "huile d'olive", 'lait', 'pain de mie', 'ail', 'tomate', 'chair à saucisse', 'oignon', 'échalote', 'herbes', 'riz blanc']}
{'_id': ObjectId('60888cb290722cacd3a9f4ba'), 'title': 'La vraie Tartiflette', 'ingredient': ['poivre', 'sel', 'ail', 'huile', 'lardons fumés', 'pomme de terre', 'oignon', 'reblochon']}
{'_id': ObjectId('60888cb290722cacd3a9f4bb'), 'title': 'Blanquette de veau facile', 'ingredient': ['poivre', 'sel', 'vin blanc', 'farine', 'citron', 'champignon', 'carotte', 'bouillon de poule', 'cube de bouillon', 'blanquette de veau', 'oignon jaune', 'crème fraîche', "jaune d'oeuf"]}
{'_id': ObjectId('60888cb290722cacd3a9f4bc'), 'title': 'Gratin Dauphinois', 'ingredient': ['poivre', 'sel', 

In [7]:
def searcheAllergie(db, a):
    liste = []
    for res in db:
        for j in res.find("ingredient"):
            if(j.find() != a ):
                liste.append(res.find('title'))
        
    return liste
u = 'poivre'    
test = searcheAllergie(cur, u)

print(test)

[]


In [8]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [9]:
import streamlit as st


scrap.ipynb
